In [1]:
from itertools import chain
from nfdk.model_v2.simple_model_pojo_pool import SimpleModelPojoPool
from nfdk.pojos.model import InventoryItemTypeEnum, LinkLayerEnum, InventoryItemRef
from nfdk.utils.brain_rest_connector import BrainRestConnector
import asyncio

In [2]:
HOST = '192.168.33.10'
PORT = 8080
pool = SimpleModelPojoPool.create_with_rest_connector_unsecured(host=HOST, port=PORT, fail_on_missing_pojo=False)
brain_rest_connector = BrainRestConnector.create_non_secured(host=HOST, port=PORT)

In [3]:
async def run():
    await asyncio.gather(pool.connector.fetch_inventory(itype=InventoryItemTypeEnum.ROUTER),
                         pool.connector.fetch_links(layer=LinkLayerEnum.LSP),
                         pool.connector.fetch_links(layer=LinkLayerEnum.IGP),
                         pool.connector.fetch_paths(),
                         pool.connector.fetch_sites()
                         )
    pool.resolve_refs()
    return

async def get_lsps(router, layers):
    link_lists = [await brain_rest_connector.get_links(layer, root=router.guid) for layer in layers]
    return [link for link in chain(*link_lists)]

async def get_routers():
    router_list = [await brain_rest_connector.get_inventory(itype=InventoryItemTypeEnum.ROUTER)]
    return [router for router in chain(*router_list)]

async def clean_up():
    await pool.connector.close()
    await brain_rest_connector.close_connection()

In [5]:
# Fill the pool and resolv references
%time asyncio.get_event_loop().run_until_complete(run())

CPU times: user 2.65 s, sys: 134 ms, total: 2.79 s
Wall time: 6.01 s


In [84]:
all_routers = %time asyncio.get_event_loop().run_until_complete(get_routers())

CPU times: user 123 ms, sys: 154 ms, total: 277 ms
Wall time: 355 ms


In [109]:
lsp_path_hops = []
for i, router in enumerate(all_routers):
    router_lsps = asyncio.get_event_loop().run_until_complete(get_lsps(router, [LinkLayerEnum.LSP]))
    for lsp in router_lsps:
        if lsp.paths:
            lsp_path_guid = [x.guid for x in lsp.paths]
            lsp_path = pool.get_by_guid(lsp_path_guid[0])
            path_hops = [hop for hop in lsp_path.hops]
            lsp_path_hops.append([router.name, router.management_ip, lsp.name, lsp.guid, lsp_path.guid, len(path_hops)])
#             print(router.name, lsp_path.guid, len(path_hops))

    
#     lsp_by_ip[router.management_ip] = sorted([lsp.name for lsp in router_lsps])
    
    
    
# import pickle
# with open('lsp_by_ip.pickle', 'wb') as r_hash:
#     pickle.dump(lsp_by_ip, r_hash)
# import pickle
# with open('lsp_by_ip.pickle', 'rb') as r_hash:
#     lsp_by_ip = pickle.load(r_hash)


In [116]:
[x for x in lsp_path_hops if x[5] > 24]

[['HER-R10-EA',
  '10.190.0.164',
  'To-CUN-R2-EA',
  'LI/lsp/10.190.0.164/10.190.3.34/To-CUN-R2-EA',
  'PA/lsp/10.190.0.164/10.190.3.34/To-CUN-R2-EA',
  25],
 ['CUN-R2-EA',
  '10.190.3.34',
  'To-CUN-R2-EA',
  'LI/lsp/10.190.0.164/10.190.3.34/To-CUN-R2-EA',
  'PA/lsp/10.190.0.164/10.190.3.34/To-CUN-R2-EA',
  25]]

In [78]:
for x in lsp_by_ip['10.190.1.151']:
    print(x)
    break

To-GDL-CB-2


In [79]:
# Router will have been selected from a list or entered manually
# Get the LSPs associated with a router GUID
# This is your example router TIJ-M5-EA
router_guid = 'IN/10.190.0.56'
router = pool.get_by_guid(router_guid)
%time router_lsps = asyncio.get_event_loop().run_until_complete(get_lsps(router, [LinkLayerEnum.LSP]))

CPU times: user 11.2 ms, sys: 4.75 ms, total: 16 ms
Wall time: 26.3 ms


In [80]:
router_lsps[53]

{
    "activeProtectionPriority": null,
    "bidi": false,
    "desc": null,
    "distanceMeters": null,
    "extra": null,
    "guid": "LI/lsp/10.190.0.56/10.190.1.140/To-MTY-M4-EA",
    "holdPriority": 0,
    "latencyMicros": null,
    "layer": "LSP",
    "name": "To-MTY-M4-EA",
    "operStatus": "UP",
    "pathGroupType": "SINGLE_PATH",
    "paths": [
        {
            "guid": "PA/lsp/10.190.0.56/10.190.1.140/To-MTY-M4-EA"
        }
    ],
    "portA": {
        "guid": "PO/igp/10.190.0.56/10.190.0.56",
        "type": "IGP"
    },
    "portB": {
        "guid": "PO/igp/10.190.1.140/10.190.1.140",
        "type": "IGP"
    },
    "protectionStatus": "N_A",
    "provider": "NOKIA_CLI",
    "role": "REGULAR",
    "service": false,
    "setupPriority": 7,
    "speedBps": 0,
    "srlgs": null
}

In [81]:
lsp_name = 'MTY-M4'

In [82]:
[x.name for x in router_lsps if lsp_name in x.name]

['To-MTY-M4-EA', 'To-MTY-M4-EB']

In [55]:
lsp

[]

In [83]:
# List of all the LSP for the selected router
[x.guid for x in router_lsps]
print(f'Router {router_guid} has {len(router_lsps)} LSPs')

Router IN/10.190.0.56 has 190 LSPs


In [67]:
# Finding your example LSP name To-MTY-M4-EA
for i, lsp in enumerate(router_lsps):
    if lsp.name == 'To-MTY-M4-EA':
        print(i)

In [8]:
# This is where the LSP would be selected from a list, entered or a hyper-link selection made
lsp = router_lsps[53]

In [9]:
if lsp.paths:
    lsp_path_guid = [x.guid for x in lsp.paths]
    lsp_path = pool.get_by_guid(lsp_path_guid[0])

    row = []
    for path_hop in lsp_path.hops:
        temp = path_hop.link_non_throwing.guid.split('/')
        if path_hop.direction.name == "B_TO_A":
            row.append([temp[4], temp[5]])
            row.append([temp[2], temp[3]])
        if path_hop.direction.name == "A_TO_B":
            row.append([temp[2], temp[3]])
            row.append([temp[4], temp[5]])

    print(f'Router {router.name} ({router.guid}) sees {len(router_lsps)} LSPs')
    print(f'LSP Name: {lsp.name} \nStatus: {lsp.oper_status.name} \nLSP Path GUID: {lsp_path_guid[0]}')
    print(f'\n{"Site Name":30} {"Router Name":30} {"Router Mgt. IP":20} {"IGP IP":20}')
    print(f'{"-" * 30} {"-" * 30} {"-" * 20}  {"-" * 20}')
    for line in row:
        router = pool.get_by_guid(f'IN/{line[0]}')
        site = router.site_non_throwing
        print(f'{site.name:30} {router.name:30} {line[0]:20} {line[1]:20}')
else:
    print(f'There are no paths for LSP {lsp.name}')

Router TIJ-M5-EA (IN/10.190.0.56) sees 190 LSPs
LSP Name: To-MTY-M4-EA 
Status: UP 
LSP Path GUID: PA/lsp/10.190.0.56/10.190.1.140/To-MTY-M4-EA

Site Name                      Router Name                    Router Mgt. IP       IGP IP              
------------------------------ ------------------------------ --------------------  --------------------
RSO TIJUANA                    TIJ-M5-EA                      10.190.0.56          10.190.33.121       
RSO TIJUANA                    TIJ-M5-CA                      10.190.0.54          10.190.33.120       
RSO TIJUANA                    TIJ-M5-CA                      10.190.0.54          10.190.33.113       
BCN-1015 PORFIRIO DIAZ         TIJMTX-BCN1015-CA              10.190.0.0           10.190.33.112       
BCN-1015 PORFIRIO DIAZ         TIJMTX-BCN1015-CA              10.190.0.0           10.190.33.134       
JUA-002 SWITCH                 JUAMTX-JUA002-CA               10.190.0.221         10.190.33.135       
JUA-002 SWITCH        

# _more detailed output..._

In [10]:
lsp_path_guid = [x.guid for x in lsp.paths]
print(lsp_path_guid)

['PA/lsp/10.190.0.56/10.190.1.140/To-MTY-M4-EA']


In [11]:
lsp_path = pool.get_by_guid(lsp_path_guid[0])
print(lsp_path)

{
    "extra": null,
    "guid": "PA/lsp/10.190.0.56/10.190.1.140/To-MTY-M4-EA",
    "hops": [
        {
            "direction": "B_TO_A",
            "extra": null,
            "label": null,
            "link": {
                "guid": "LI/igp/10.190.0.54/10.190.33.120/10.190.0.56/10.190.33.121",
                "layer": "IGP"
            }
        },
        {
            "direction": "B_TO_A",
            "extra": null,
            "label": null,
            "link": {
                "guid": "LI/igp/10.190.0.0/10.190.33.112/10.190.0.54/10.190.33.113",
                "layer": "IGP"
            }
        },
        {
            "direction": "A_TO_B",
            "extra": null,
            "label": null,
            "link": {
                "guid": "LI/igp/10.190.0.0/10.190.33.134/10.190.0.221/10.190.33.135",
                "layer": "IGP"
            }
        },
        {
            "direction": "A_TO_B",
            "extra": null,
            "label": null,
            "link"

In [12]:
for path_hop in lsp_path.hops:
    print(f'\n{path_hop.link_non_throwing}')


{
    "activeProtectionPriority": null,
    "advRouters": [
        "10.190.0.54",
        "10.190.0.56"
    ],
    "area": null,
    "bidi": true,
    "desc": null,
    "distanceMeters": null,
    "extra": null,
    "guid": "LI/igp/10.190.0.54/10.190.33.120/10.190.0.56/10.190.33.121",
    "latencyMicros": null,
    "layer": "IGP",
    "name": "10.190.33.120 to 10.190.33.121",
    "operStatus": "UP",
    "pathGroupType": "SINGLE_PATH",
    "paths": [
        {
            "guid": "PA/igp/10.190.0.54/10.190.33.120/10.190.0.56/10.190.33.121"
        }
    ],
    "portA": {
        "guid": "PO/igp/10.190.0.54/10.190.33.120",
        "type": "IGP"
    },
    "portB": {
        "guid": "PO/igp/10.190.0.56/10.190.33.121",
        "type": "IGP"
    },
    "protectionStatus": "N_A",
    "provider": "NOKIA_CLI",
    "role": "REGULAR",
    "service": false,
    "srlgs": null
}

{
    "activeProtectionPriority": null,
    "advRouters": [
        "10.190.0.54",
        "10.190.0.0"
    ],
    "ar

In [13]:
asyncio.get_event_loop().run_until_complete(clean_up())

In [ ]:
# if the root= isn't available in your NDFK use this to get the LSPs
def test_lsp(router):
    lo_routers = []
    router = pool.get_by_guid(router_guid)
    for l in lsp:
        if router.management_ip in l.port_a_non_throwing.guid or router.management_ip in l.port_b_non_throwing.guid:
            lo_routers.append(l)
    return lo_routers

%time test = test_lsp('IN/10.190.0.103')
print(len(test))